# Machine Learning - TP2: Detectando lugares del mundo

### Dependencias

In [ ]:
# un poco menos de warnings de tensorflow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# de python, para especificar rutas de archivos y directorios
from pathlib import Path

import matplotlib.pyplot as plt
# lib para trabajar con arrays
import numpy as np

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.applications import VGG16


### Origen de datos

In [ ]:
# Fede: /media/federico/secondary/Repositorios/UCSE/MachineLearning/data_tp2/
# Andy: C:/Users/andru/Downloads/ucse-ia-2024-tp-2-clasificacion-de-imagenes/
BASE_DIR = 'C:/Users/andru/Downloads/ucse-ia-2024-tp-2-clasificacion-de-imagenes/'

TRAIN_DIR = Path(BASE_DIR + 'train')
TEST_DIR = Path(BASE_DIR + 'test')

### Variables Globales

In [ ]:
CLASES = "street", "sea", "mountain", "glacier", "forest", "buildings"
SIZE = 32

In [ ]:
images_reader = ImageDataGenerator(
    rescale=1/255,
    rotation_range=10,
    #width_shift_range=0.3,
    #height_shift_range=0.3,
    brightness_range=(0.5, 1.5),
    #horizontal_flip=True,
    #vertical_flip=True,
)

READ_PARAMS = dict(
    class_mode="categorical",  # tenemos N labels, queremos tuplas de 0s y 1s indicando cuál de los labels es
    classes=CLASES,  # para usar el mismo orden en todos lados
    target_size=(SIZE, SIZE),  # para que corra más rápido, vamos a achicar las imágenes
    color_mode="rgb",  # queremos trabajar con las imágenes a color
)

In [ ]:
train = images_reader.flow_from_directory(TRAIN_DIR, **READ_PARAMS)

In [ ]:
def sample_images(dataset):
    plt.figure(figsize=(10, 10))
    images, labels = next(dataset)
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title(CLASES[np.argmax(labels[i])])
        plt.axis("off")

In [ ]:
sample_images(train)

In [ ]:
for x_batch, y_batch in train:
    for i in range(4):
        plt.subplot(2, 2, i+1)
        plt.imshow(x_batch[i])
        plt.title(f"Class: {CLASES[i]}")
        plt.axis('off')
    plt.show()

    mean = np.mean(x_batch, axis=(0, 1, 2))
    std = np.std(x_batch, axis=(0, 1, 2))

    plt.figure(figsize=(8, 4))
    channels = ['R', 'G', 'B']
    
    plt.bar(channels, mean, color='skyblue', label='Media')
    plt.errorbar(channels, mean, yerr=std, fmt='o', color='red', label='Desviación Estándar', capsize=5)
    plt.ylabel("Valor")
    plt.title("Media y Desviación Estándar por Canal")
    plt.legend()
    plt.show()
    break

In [ ]:
contador_imagenes = {}
for subcarpeta, _, _ in os.walk(TRAIN_DIR):
    imagenes = tf.io.gfile.glob(os.path.join(subcarpeta, "*.jpg"))
    contador_imagenes[subcarpeta] = len(imagenes)

clasesplt = []
cantidades = []

for clase, cantidad in contador_imagenes.items():
        partes = clase.split('\\')
        clases = partes[-1].split(".")[0]      
        clasesplt.append(clases)
        cantidades.append(cantidad)

clasesplt.pop(0)
cantidades.pop(0)
plt.figure(figsize=(10, 6))
plt.bar(clasesplt, cantidades, color='skyblue')
plt.xlabel('Clases')
plt.ylabel('Cantidad de Imágenes')
plt.title('Cantidad de Imágenes por Clase')
plt.xticks(rotation=45)
plt.show()